# Домашна задача 4
## Обработка на природните јазици 2018/2019
### Андреј Јанчевски 151003

In [1]:
import pandas as pd
import numpy as np
import scipy
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from bs4 import BeautifulSoup
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download("punkt")
nltk.download('stopwords')
stops = set(stopwords.words('english'))
from nltk import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import cython
from gensim.models import word2vec
import re
import logging
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bani5\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bani5\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bani5\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\bani5\AppData\Roaming\Python\Python37\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
train = pd.read_csv(
    "labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
print(train.shape)
print(train.columns.values)
print(train["review"][0])
train_labels = train["sentiment"]

(25000, 3)
['id' 'sentiment' 'review']
"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actu

In [3]:
example1 = BeautifulSoup(train["review"][0])
example1.get_text()

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 2

In [4]:
def review_to_words(raw_review,
                    remove_stops=False,
                    lemmatize=False,
                    stem=False):
    review = BeautifulSoup(raw_review).get_text()
    filtered = re.sub('\d+', 'NUM', review)
    filtered = re.sub('\.', ' \. ', filtered)
    filtered = re.sub('\,', ' \, ', filtered)
    filtered = re.sub('\?', ' \? ', filtered)
    filtered = re.sub('!', ' ! ', filtered)
    filtered = re.sub('\:', ' \: ', filtered)
    filtered = re.sub('\-', ' \- ', filtered)
    filtered = re.sub('[^A-Za-z\.\,\?!\:\-]', ' ', filtered)
    filtered = filtered.lower()
    words = [token for token in filtered.split(" ") if token != '']
    if remove_stops:
        words = [word for word in words if word not in stops]
    if lemmatize:
        lemmatizer = WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word, pos='v') for word in words]
    if stem:
        ps = PorterStemmer()
        words = [ps.stem(word) for word in words]
    return words


" ".join(review_to_words(train["review"][0], lemmatize=True))

'with all this stuff go down at the moment with mj i ve start listen to his music , watch the odd documentary here and there , watch the wiz and watch moonwalker again . maybe i just want to get a certain insight into this guy who i think be really cool in the eighties just to maybe make up my mind whether he be guilty or innocent . moonwalker be part biography , part feature film which i remember go to see at the cinema when it be originally release . some of it have subtle message about mj s feel towards the press and also the obvious message of drug be bad m kay . visually impressive but of course this be all about michael jackson so unless you remotely like mj in anyway then you be go to hate this and find it bore . some may call mj an egotist for consent to the make of this movie but mj and most of his fan would say that he make it for the fan which if true be really nice of him . the actual feature film bite when it finally start be only on for num minutes or so exclude the smoot

In [5]:
def get_clean_reviews(dataframe,
                      remove_stops=False,
                      lemmatize=False,
                      stem=False):
    num_reviews = dataframe["review"].size
    clean_reviews = []
    for i in range(0, num_reviews):
        if (i + 1) % 1000 == 0:
            print("Review " + str(i + 1) + " of " + str(num_reviews) + "\n")
        clean_reviews.append(" ".join(
            review_to_words(dataframe["review"][i], remove_stops, lemmatize,
                            stem)))
    return clean_reviews


clean_train_reviews = get_clean_reviews(train, lemmatize=True)

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [6]:
vectorizer = CountVectorizer(
    analyzer="word",
    tokenizer=None,
    preprocessor=None,
    stop_words=None,
    max_features=5000)


def get_bow_dataset(clean_reviews, test=False):
    if test:
        features = vectorizer.transform(clean_reviews)
    else:
        features = vectorizer.fit_transform(clean_reviews)
    return features.toarray()


train_bow_dataset = get_bow_dataset(clean_train_reviews)
train_bow_dataset.shape

(25000, 5000)

In [7]:
vocab = vectorizer.get_feature_names()
dist = np.sum(train_bow_dataset, axis=0)

for tag, count in zip(vocab, dist):
    print(count, tag)

272 abandon
125 abc
108 abilities
454 ability
1259 able
17372 about
817 above
85 abraham
73 abrupt
116 absence
83 absent
352 absolute
1485 absolutely
153 absorb
306 absurd
76 absurdity
299 abuse
91 abusive
98 abysmal
297 academy
704 accent
575 accept
130 acceptable
71 acceptance
93 access
318 accident
200 accidentally
118 acclaim
185 accompany
223 accomplish
306 accord
262 account
81 accuracy
284 accurate
163 accuse
75 ace
405 achieve
124 achievement
90 acid
95 acknowledge
97 acquire
971 across
8795 act
3666 action
75 active
83 activities
2389 actor
4486 actors
1219 actress
369 actresses
793 actual
4237 actually
143 ad
302 adam
98 adams
235 adapt
453 adaptation
80 adaptations
1752 add
163 addict
347 addition
74 additional
183 address
113 adequate
73 admirable
212 admire
738 admit
134 admittedly
76 adolescent
138 adopt
101 adorable
109 adore
510 adult
376 adults
247 advance
171 advantage
717 adventure
171 advertise
259 advice
165 advise
346 affair
71 affairs
312 affect
73 affection
130 

limit_output extension: Maximum message size of 10000 exceeded with 12741 characters

In [131]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
naive_bayes = GaussianNB()
knn = KNeighborsClassifier(
    n_neighbors=20,
    weights="distance",
    algorithm="auto",
    metric="cosine",
    n_jobs=-1)
svm = SVC(
    C=1e8,
    kernel='rbf',
    gamma="scale",
    decision_function_shape="ovr",
    class_weight="balanced")
logistic_regression = LogisticRegression(
    C=1e8,
    solver='newton-cg',
    max_iter=1000,
    class_weight="balanced",
    n_jobs=-1)
random_forest = RandomForestClassifier(
    n_estimators=1000,
    criterion="entropy",
    min_samples_leaf=5,
    class_weight="balanced",
    n_jobs=-1)
neural_network = MLPClassifier(
    hidden_layer_sizes=(200, 100),
    activation="tanh",
    solver="adam",
    learning_rate="adaptive",
    learning_rate_init=0.001,
    max_iter=1000,
    early_stopping=True,
    validation_fraction=0.1)

In [132]:
from sklearn.metrics import roc_auc_score


def train_and_score_classifiers(train_dataset, train_labels):
    rows, cols = train_dataset.shape
    validation_dataset = train_dataset[int(0.8 * rows):]
    validation_labels = train_labels[int(0.8 * rows):]
    train_dataset = train_dataset[:int(0.8 * rows)]
    train_labels = train_labels[:int(0.8 * rows)]

    print(train_dataset.shape)
    print(validation_dataset.shape)

    scores = {}

    naive_bayes.fit(train_dataset, train_labels)
    predicted_labels = naive_bayes.predict(validation_dataset)
    scores[naive_bayes] = roc_auc_score(validation_labels, predicted_labels)
    print("Naive Bayes done")

    knn.fit(train_dataset, train_labels)
    predicted_labels = knn.predict(validation_dataset)
    scores[knn] = roc_auc_score(validation_labels, predicted_labels)
    print("KNN done")

    #logistic_regression.fit(train_dataset, train_labels)
    #predicted_labels = logistic_regression.predict(validation_dataset)
    #scores[logistic_regression] = roc_auc_score(validation_labels,
    #                                            predicted_labels)
    #print("Logistic Regression done")

    #svm.fit(train_dataset, train_labels)
    #predicted_labels = svm.predict(validation_dataset)
    #scores[svm] = roc_auc_score(validation_labels, predicted_labels)
    #print("SVM done")

    neural_network.fit(train_dataset, train_labels)
    predicted_labels = neural_network.predict(validation_dataset)
    scores[neural_network] = roc_auc_score(validation_labels, predicted_labels)
    print("Neural Network done")

    random_forest.fit(train_dataset, train_labels)
    predicted_labels = random_forest.predict(validation_dataset)
    scores[random_forest] = roc_auc_score(validation_labels, predicted_labels)
    print("Random Forest done")

    print(scores.values())

    best_model = max(scores.items(), key=lambda x: x[1])
    best_model = best_model[0]
    return best_model

In [144]:
best_model = train_and_score_classifiers(train_bow_dataset, train_labels)
best_model

(20000, 5000)
(5000, 5000)
Naive Bayes done
KNN done
Neural Network done
Random Forest done
dict_values([0.7285451487034533, 0.6709788875916595, 0.8691195823604112, 0.8532225482364508])


MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(200, 100), learning_rate='adaptive',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [11]:
test = pd.read_csv("testData.tsv", header=0, delimiter="\t", quoting=3)
print(test.shape)
clean_test_reviews = get_clean_reviews(test, lemmatize=True)
test_bow_dataset = get_bow_dataset(clean_test_reviews, test=True)

(25000, 2)
Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [79]:
def generate_submission(submission_name, model, test_dataframe, test_dataset):
    result = model.predict(test_dataset)
    output = pd.DataFrame(data={
        "id": test_dataframe["id"],
        "sentiment": result
    })
    output.to_csv(submission_name, index=False, quoting=3)

In [13]:
generate_submission("Andrej_Janchevski.csv", best_model, test,
                    test_bow_dataset)

In [14]:
unlabeled_train = pd.read_csv(
    "unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
unlabeled_train.shape

(50000, 2)

In [15]:
clean_unlabeled_train_reviews = get_clean_reviews(
    unlabeled_train, lemmatize=True)

Review 1000 of 50000

Review 2000 of 50000

Review 3000 of 50000

Review 4000 of 50000

Review 5000 of 50000

Review 6000 of 50000

Review 7000 of 50000

Review 8000 of 50000

Review 9000 of 50000

Review 10000 of 50000

Review 11000 of 50000

Review 12000 of 50000

Review 13000 of 50000

Review 14000 of 50000

Review 15000 of 50000

Review 16000 of 50000

Review 17000 of 50000

Review 18000 of 50000

Review 19000 of 50000

Review 20000 of 50000

Review 21000 of 50000

Review 22000 of 50000

Review 23000 of 50000

Review 24000 of 50000

Review 25000 of 50000

Review 26000 of 50000

Review 27000 of 50000

Review 28000 of 50000

Review 29000 of 50000

Review 30000 of 50000

Review 31000 of 50000

Review 32000 of 50000

Review 33000 of 50000

Review 34000 of 50000

Review 35000 of 50000

Review 36000 of 50000

Review 37000 of 50000

Review 38000 of 50000

Review 39000 of 50000

Review 40000 of 50000

Review 41000 of 50000

Review 42000 of 50000

Review 43000 of 50000

Review 44000 of 5000

In [16]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')


def review_to_sentences(review,
                        remove_stops=False,
                        lemmatize=False,
                        stem=False):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(
                review_to_words(raw_sentence, remove_stops, lemmatize, stem))
    return sentences


review_to_sentences(train["review"][0], lemmatize=True)

[['with',
  'all',
  'this',
  'stuff',
  'go',
  'down',
  'at',
  'the',
  'moment',
  'with',
  'mj',
  'i',
  've',
  'start',
  'listen',
  'to',
  'his',
  'music',
  ',',
  'watch',
  'the',
  'odd',
  'documentary',
  'here',
  'and',
  'there',
  ',',
  'watch',
  'the',
  'wiz',
  'and',
  'watch',
  'moonwalker',
  'again',
  '.'],
 ['maybe',
  'i',
  'just',
  'want',
  'to',
  'get',
  'a',
  'certain',
  'insight',
  'into',
  'this',
  'guy',
  'who',
  'i',
  'think',
  'be',
  'really',
  'cool',
  'in',
  'the',
  'eighties',
  'just',
  'to',
  'maybe',
  'make',
  'up',
  'my',
  'mind',
  'whether',
  'he',
  'be',
  'guilty',
  'or',
  'innocent',
  '.'],
 ['moonwalker',
  'be',
  'part',
  'biography',
  ',',
  'part',
  'feature',
  'film',
  'which',
  'i',
  'remember',
  'go',
  'to',
  'see',
  'at',
  'the',
  'cinema',
  'when',
  'it',
  'be',
  'originally',
  'release',
  '.'],
 ['some',
  'of',
  'it',
  'have',
  'subtle',
  'message',
  'about',
  'm

In [17]:
sentences = []

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, lemmatize=True)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, lemmatize=True)

Parsing sentences from training set


C:\Users\bani5\AppData\Roaming\Python\Python37\site-packages\bs4\__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\bani5\AppData\Roaming\Python\Python37\site-packages\bs4\__init__.py:273: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\bani5\AppData\Roaming\Python\Python37\site-packages\bs4\__init__.py:336: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


C:\Users\bani5\AppData\Roaming\Python\Python37\site-packages\bs4\__init__.py:336: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\bani5\AppData\Roaming\Python\Python37\site-packages\bs4\__init__.py:336: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\bani5\AppData\Roaming\Python\Python37\site-packages\bs4\__init__.py:273: UserWarning: "b'... ...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful 

In [18]:
print(len(sentences))
print(sentences[0])
print(sentences[57])

795538
['with', 'all', 'this', 'stuff', 'go', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'start', 'listen', 'to', 'his', 'music', ',', 'watch', 'the', 'odd', 'documentary', 'here', 'and', 'there', ',', 'watch', 'the', 'wiz', 'and', 'watch', 'moonwalker', 'again', '.']
['my', 'god', ',', 'what', 'a', 'brilliant', 'experience', '.']


In [22]:
num_features = 500  # Word vector dimensionality
min_word_count = 50  # Minimum word count
num_workers = 4  # Number of threads to run in parallel
context = 10  # Context window size
downsampling = 1e-3  # Downsample setting for frequent words

print("Training model...")
model = word2vec.Word2Vec(
    sentences,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context,
    sample=downsampling)
word2vec.Word2Vec()
model.init_sims(replace=True)

model_name = "word2vec.model"
model.save(model_name)

2018-12-20 00:11:52,041 : INFO : collecting all words and their counts
2018-12-20 00:11:52,042 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-12-20 00:11:52,082 : INFO : PROGRESS: at sentence #10000, processed 254633 words, keeping 14748 word types
2018-12-20 00:11:52,121 : INFO : PROGRESS: at sentence #20000, processed 509007 words, keeping 20656 word types
2018-12-20 00:11:52,160 : INFO : PROGRESS: at sentence #30000, processed 756401 words, keeping 24865 word types
2018-12-20 00:11:52,198 : INFO : PROGRESS: at sentence #40000, processed 1011564 words, keeping 28468 word types
2018-12-20 00:11:52,234 : INFO : PROGRESS: at sentence #50000, processed 1259395 words, keeping 31366 word types


Training model...


2018-12-20 00:11:52,273 : INFO : PROGRESS: at sentence #60000, processed 1508834 words, keeping 33869 word types
2018-12-20 00:11:52,313 : INFO : PROGRESS: at sentence #70000, processed 1760225 words, keeping 36126 word types
2018-12-20 00:11:52,351 : INFO : PROGRESS: at sentence #80000, processed 2007022 words, keeping 38188 word types
2018-12-20 00:11:52,390 : INFO : PROGRESS: at sentence #90000, processed 2259952 words, keeping 40270 word types
2018-12-20 00:11:52,427 : INFO : PROGRESS: at sentence #100000, processed 2510143 words, keeping 42040 word types
2018-12-20 00:11:52,465 : INFO : PROGRESS: at sentence #110000, processed 2758616 words, keeping 43669 word types
2018-12-20 00:11:52,505 : INFO : PROGRESS: at sentence #120000, processed 3008744 words, keeping 45473 word types
2018-12-20 00:11:52,544 : INFO : PROGRESS: at sentence #130000, processed 3263445 words, keeping 46988 word types
2018-12-20 00:11:52,582 : INFO : PROGRESS: at sentence #140000, processed 3503763 words, kee

2018-12-20 00:11:55,104 : INFO : PROGRESS: at sentence #780000, processed 19672156 words, keeping 109008 word types
2018-12-20 00:11:55,146 : INFO : PROGRESS: at sentence #790000, processed 19927791 words, keeping 109653 word types
2018-12-20 00:11:55,167 : INFO : collected 110074 word types from a corpus of 20066678 raw words and 795538 sentences
2018-12-20 00:11:55,168 : INFO : Loading a fresh vocabulary
2018-12-20 00:11:55,217 : INFO : effective_min_count=50 retains 12067 unique words (10% of original 110074, drops 98007)
2018-12-20 00:11:55,218 : INFO : effective_min_count=50 leaves 19526244 word corpus (97% of original 20066678, drops 540434)
2018-12-20 00:11:55,245 : INFO : deleting the raw counts dictionary of 110074 items
2018-12-20 00:11:55,249 : INFO : sample=0.001 downsamples 50 most-common words
2018-12-20 00:11:55,250 : INFO : downsampling leaves estimated 13228187 word corpus (67.7% of prior 19526244)
2018-12-20 00:11:55,279 : INFO : estimated required memory for 120

limit_output extension: Maximum message size of 10000 exceeded with 10044 characters

In [23]:
model.wv.doesnt_match("man woman child kitchen".split())

C:\Users\bani5\AppData\Roaming\Python\Python37\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'kitchen'

In [25]:
model.wv.doesnt_match("france england germany berlin".split())

C:\Users\bani5\AppData\Roaming\Python\Python37\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'berlin'

In [26]:
model.wv.doesnt_match("paris berlin london austria".split())

C:\Users\bani5\AppData\Roaming\Python\Python37\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'austria'

In [29]:
model.wv.most_similar("man")

C:\Users\bani5\AppData\Roaming\Python\Python37\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('woman', 0.6220428347587585),
 ('lad', 0.5684807300567627),
 ('lady', 0.5563461780548096),
 ('monk', 0.5445902943611145),
 ('millionaire', 0.5421693325042725),
 ('businessman', 0.5183982253074646),
 ('priest', 0.5182092189788818),
 ('sailor', 0.5145820379257202),
 ('men', 0.5050533413887024),
 ('person', 0.5049101114273071)]

In [30]:
model.wv.most_similar("queen")

C:\Users\bani5\AppData\Roaming\Python\Python37\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('princess', 0.6480422019958496),
 ('stepmother', 0.5759179592132568),
 ('maid', 0.572234570980072),
 ('nun', 0.5567984580993652),
 ('belle', 0.5451830625534058),
 ('goddess', 0.5433132648468018),
 ('regina', 0.5393650531768799),
 ('widow', 0.5368809103965759),
 ('prince', 0.5348613858222961),
 ('victoria', 0.5343594551086426)]

In [39]:
model.wv.most_similar("awful")

C:\Users\bani5\AppData\Roaming\Python\Python37\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('terrible', 0.7406212687492371),
 ('dreadful', 0.7247092723846436),
 ('atrocious', 0.718103289604187),
 ('horrible', 0.7075750231742859),
 ('horrendous', 0.6855326890945435),
 ('abysmal', 0.676424503326416),
 ('horrid', 0.6581128835678101),
 ('lousy', 0.6520488262176514),
 ('amateurish', 0.6165722608566284),
 ('appal', 0.6091347336769104)]

In [43]:
model.wv.most_similar("!")

[('lol', 0.5132268667221069),
 ('wow', 0.48045819997787476),
 ('duh', 0.46121692657470703),
 ('friggin', 0.4604405164718628),
 ('ugh', 0.44410884380340576),
 ('omg', 0.4373909533023834),
 ('yay', 0.43644386529922485),
 ('stink', 0.42847511172294617),
 ('gosh', 0.4257442057132721),
 ('whoa', 0.42529821395874023)]

In [60]:
model.wv.most_similar("cat")

C:\Users\bani5\AppData\Roaming\Python\Python37\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('dog', 0.6666625738143921),
 ('mouse', 0.6510213017463684),
 ('pet', 0.6108667254447937),
 ('sheep', 0.5850355625152588),
 ('rabbit', 0.5639858245849609),
 ('bird', 0.5473445653915405),
 ('puppy', 0.5416003465652466),
 ('mice', 0.5400309562683105),
 ('pig', 0.5356342196464539),
 ('puppies', 0.5168529748916626)]

In [53]:
model.trainables.syn1neg.shape

(12067, 500)

In [55]:
model.wv["flower"]

array([ 0.04051628, -0.02039886,  0.01202334, -0.02606667, -0.02133823,
       -0.10017185, -0.01272653, -0.09540585, -0.18299694,  0.16361529,
       -0.02757043,  0.03140888,  0.04576864,  0.00403805,  0.01216199,
       -0.03088489,  0.00419512, -0.03025832,  0.0387293 , -0.00183426,
       -0.07627061,  0.02369102,  0.01339156, -0.00823164, -0.06077476,
       -0.01229963,  0.08358763, -0.03346688,  0.03895821, -0.05717836,
       -0.02232922,  0.07010701,  0.02244833, -0.0275238 , -0.00862076,
        0.03182406,  0.08041867, -0.09431239, -0.09653579,  0.03285069,
       -0.06863715,  0.05598413, -0.02643139, -0.04322631,  0.04394339,
        0.04422955,  0.03573749,  0.01240978,  0.02298628, -0.07040206,
        0.01113935,  0.00320343,  0.00068471,  0.03807371, -0.02902559,
       -0.00827401,  0.02835255,  0.01023153, -0.08672807, -0.05440461,
       -0.00932875,  0.06281601,  0.05953036,  0.00314489,  0.04248114,
        0.01662689,  0.02260633,  0.04781896,  0.03756152,  0.01

In [61]:
word2vec_vocab = set(model.wv.index2word)
word2vec_vocab_size = len(word2vec_vocab)
print(word2vec_vocab_size)

12067


'watch time chasers , it obvious that it be make by a bunch of friends . maybe they be sit around one day in film school and say , hey , let s pool our money together and make a really bad movie ! or something like that . what ever they say , they still end up make a really bad movie - - dull story , bad script , lame act , poor cinematography , bottom of the barrel stock music , etc . all corner be cut , except the one that would have prevent this film s release . life s like that .'

In [63]:
def get_average_word2vec(review):
    featureVec = np.zeros((num_features, ), dtype="float32")
    for word in review.split(" "):
        if word in word2vec_vocab:
            featureVec = np.add(featureVec, model.wv[word])
    featureVec = np.divide(featureVec, word2vec_vocab_size)
    return featureVec


get_average_word2vec(clean_unlabeled_train_reviews[0])

array([ 1.05879681e-05, -3.56698147e-05,  1.42450499e-05,  5.55367878e-05,
        5.09753081e-05,  1.00684832e-04, -1.84193341e-04, -1.16128103e-04,
        1.93949862e-04, -1.08226159e-04,  5.17251065e-05, -1.91901404e-06,
        2.11922816e-05, -5.91590615e-05,  5.28755590e-05, -4.60627552e-06,
       -6.18077029e-05,  2.07775906e-06,  1.25279970e-04,  6.01875536e-06,
       -1.54932088e-04,  1.57997220e-05,  1.46106759e-05, -2.09213886e-05,
       -4.31003136e-05, -8.99111401e-05, -7.77515088e-05,  1.39626500e-04,
       -5.35250438e-05, -1.22633619e-05, -6.97180221e-05,  3.76372118e-05,
        2.99072873e-08, -1.56479859e-04,  6.04848792e-05,  1.25872350e-04,
        6.36449331e-05, -3.65261212e-06, -3.44713480e-05, -9.35974385e-05,
       -2.07267221e-04,  5.52353813e-05,  4.13939342e-05,  4.47626417e-06,
        1.38951204e-04,  4.77079157e-05,  4.62642383e-05,  5.05216376e-05,
        1.47593993e-04,  8.73936297e-05,  9.80005352e-05, -1.00811121e-04,
       -1.25117600e-04,  

In [118]:
from sklearn.cluster import KMeans
k_means = KMeans(
    n_clusters=word2vec_vocab_size // 5,
    n_init=5,
    max_iter=1000,
    verbose=True,
    tol=1e-6,
    n_jobs=-1)
cluster_index = k_means.fit_predict(model.trainables.syn1neg)

Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 18264.07
start iteration
done sorting
end inner loop
Iteration 1, inertia 17848.867
start iteration
done sorting
end inner loop
Iteration 2, inertia 17751.45
start iteration
done sorting
end inner loop
Iteration 3, inertia 17710.213
start iteration
done sorting
end inner loop
Iteration 4, inertia 17690.688
start iteration
done sorting
end inner loop
Iteration 5, inertia 17680.547
start iteration
done sorting
end inner loop
Iteration 6, inertia 17675.209
start iteration
done sorting
end inner loop
Iteration 7, inertia 17671.895
start iteration
done sorting
end inner loop
Iteration 8, inertia 17669.697
start iteration
done sorting
end inner loop
Iteration 9, inertia 17667.318
start iteration
done sorting
end inner loop
Iteration 10, inertia 17665.924
start iteration
done sorting
end inner loop
Iteration 11, inertia 17665.156
start iteration
done sorting
end inner loop
Iteration 12, inertia 17664.318


end inner loop
Iteration 16, inertia 17693.21
start iteration
done sorting
end inner loop
Iteration 17, inertia 17693.002
start iteration
done sorting
end inner loop
Iteration 18, inertia 17692.871
start iteration
done sorting
end inner loop
Iteration 19, inertia 17692.871
center shift 0.000000e+00 within tolerance 6.681592e-09


In [120]:
word_centroid_map = dict(zip(model.wv.index2word, cluster_index))
num_centroids = max(word_centroid_map.values()) + 1

for cluster in range(0, 10):
    print("\nCluster " + str(cluster))
    words = []
    for i in range(0, len(word_centroid_map.values())):
        if (list(word_centroid_map.values())[i] == cluster):
            words.append(list(word_centroid_map.keys())[i])
    print(words)


Cluster 0
['riders', 'cowboys', 'platoon', 'commando', 'giants', 'sailors', 'fortress', 'wag', 'napoleon', 'inc', 'viking', 'helicopters', 'trench', 'elephants', 'yankee', 'rumble', 'bastards', 'cavalry', 'lions', 'tigers', 'stealth', 'ww', 'royale', 'airplanes', 'armies', 'brotherhood', 'apache', 'saber', 'yokai', 'wrestlers', 'imperial', 'commandos', 'swordplay', 'iwo', 'jima', 'tens', 'colonies', 'battleship']

Cluster 1
['fellow', 'assistant', 'staff', 'buddies', 'guests', 'frequent', 'photos', 'habit', 'pimp', 'peer', 'lunch', 'girlfriends', 'suggestion', 'colleagues', 'customers', 'colleague', 'client', 'employees', 'honeymoon', 'followers', 'assistance', 'fuss', 'employer', 'sessions', 'boyfriends', 'superiors', 'classmates', 'beers', 'clients', 'homework']

Cluster 2
['riff', 'exclusively', 'misplace', 'superfluous', 'daft', 'repetition', 'carefree', 'acquaint', 'arbitrary', 'comically', 'unrecognizable', 'ubiquitous', 'faithfully', 'anachronistic', 'fabricate', 'coarse', 'biz

In [126]:
def get_bag_of_centroids(review):
    bag_of_centroids = np.zeros(num_centroids, dtype="float32")
    for word in review.split(" "):
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    return bag_of_centroids

In [127]:
def get_word2vec_dataset(clean_reviews):
    counter = 0
    reviewFeatureVecs = np.zeros(
        (len(clean_reviews), num_features + num_centroids), dtype="float32")
    for review in clean_reviews:
        if counter % 1000 == 0:
            print("Review " + str(counter) + " of " + str(len(clean_reviews)))
        average_word2vec_part = np.array(get_average_word2vec(review))
        cluster_word2vec_part = np.array(get_bag_of_centroids(review))
        reviewFeatureVecs[counter][:num_features] = average_word2vec_part
        reviewFeatureVecs[counter][num_features:] = cluster_word2vec_part
        counter = counter + 1
    return reviewFeatureVecs

In [128]:
train_word2vec_dataset = get_word2vec_dataset(clean_train_reviews)

Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000


In [129]:
test_word2vec_dataset = get_word2vec_dataset(clean_test_reviews)

Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000


In [133]:
best_model_2 = train_and_score_classifiers(train_word2vec_dataset,
                                           train_labels)
best_model_2

(20000, 2913)
(5000, 2913)
Naive Bayes done
KNN done
Neural Network done
Random Forest done
dict_values([0.7509364374667158, 0.6518264051042562, 0.8742843902339109, 0.8161612592683626])


MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(200, 100), learning_rate='adaptive',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [134]:
generate_submission("Andrej_Janchevski_2.csv", best_model_2, test,
                    test_word2vec_dataset)

In [141]:
train_combined_dataset = np.hstack((train_bow_dataset, train_word2vec_dataset))
test_combined_dataset = np.hstack((test_bow_dataset, test_word2vec_dataset))
print(train_combined_dataset.shape)
print(test_combined_dataset.shape)

(25000, 7913)
(25000, 7913)


In [142]:
best_model_3 = train_and_score_classifiers(train_combined_dataset,
                                           train_labels)
best_model_3

(20000, 7913)
(5000, 7913)
Naive Bayes done
KNN done
Neural Network done
Random Forest done
dict_values([0.7449120017615009, 0.6624664597927165, 0.8809258633402975, 0.8201483194051861])


MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(200, 100), learning_rate='adaptive',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [143]:
generate_submission("Andrej_Janchevski_3.csv", best_model_3, test,
                    test_combined_dataset)

Bag of Words пристапот уште веднаш на почеток за ова податочно множество ни даде задоволителни резултати. <br/>
Word2Vec моделот поради малата големина на множеството даваше полоши резултати но со модификации и додавање на кластерирањето на зборовите се приближија резултатите до тие на Bag of Words. <br/>
Комбинирањето на двата модела не даде значително подобрување кај резултатите, но сепак така се добиваат најдобри резултати.